In [1]:
! pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.3 MB/s eta 0:00:00


# Student Loan Risk with Deep Learning

In [2]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
import numpy as np
import keras_tuner as kt

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"], axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_std = scaler.fit_transform(X_train)

# Fit the scaler to the features training dataset
X_test_std = scaler.fit_transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
print(input_nodes)


11


In [11]:
# Define the number of hidden nodes for the first hidden layer
nodes_layer1 = 6

# Review the number hidden nodes in the first layer
print(nodes_layer1)

# Define the number of hidden nodes for the second hidden layer
nodes_layer2 = 3

# Review the number hidden nodes in the second layer
print(nodes_layer2)

# Define the number of neurons in the output layer
nodes_output = 1

6
3


In [12]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=nodes_layer1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
model.add(Dense(units=nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=nodes_output, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Display the Sequential model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 6)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              21 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_std, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4454 - loss: 0.7487
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4772 - loss: 0.7115
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4891 - loss: 0.7036
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5671 - loss: 0.6849
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6390 - loss: 0.6739
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6820 - loss: 0.6640
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6676 - loss: 0.6510
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7219 - loss: 0.6259
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7115 - loss: 0.6076
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6932 - loss: 0.6169
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7299 - loss: 0.6009
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7260 - lo

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_std, y_test, verbose=2)
# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 24ms/step - accuracy: 0.7275 - loss: 0.5223
Loss: 0.5223473906517029, Accuracy: 0.7275000214576721


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [17]:
# Set the model's file path
path = Path("student_loans.keras")

# Export your model to a keras file
model.save(path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [18]:
# Set the model's file path
path = Path("student_loans.keras")

# Load the model to a new object
model = tf.keras.models.load_model(path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions with the test data
predictions = model.predict(X_test,verbose=2)


# Display a sample of the predictions
predictions[:5]
predictions_rounded = [round(prediction[0],0) for prediction in predictions]
predictions_rounded[:5]

13/13 - 0s - 13ms/step


[0.0, 0.0, 1.0, 0.0, 0.0]

In [20]:
# Save the predictions to a DataFrame and round the predictions to binary results
df = pd.DataFrame(predictions_rounded)
df

,0
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
395,1.0
396,0.0
397,0.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [21]:
# Print the classification report with the y test data and predictions
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_rounded))

              precision    recall  f1-score   support

           0       0.49      0.87      0.63       188
           1       0.63      0.20      0.30       212

    accuracy                           0.51       400
   macro avg       0.56      0.53      0.46       400
weighted avg       0.56      0.51      0.45       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

*Loans are made up of certain characteristics - amount, duration, rate. Instead of making a binary decision - 'Yes / No' - a model should match a students profile to a particular loan type. Higher risk students (those that would have previously been rejected) will get mapped to loans of lower dollar amounts, less time and higher rate. Lower risk sudents would get lower rates and have access to more funds. The model should be refreshed every 6 months based on the performance of the student and changes in their circumstances in order to drive accountability with lower risk students (those that receivied loans that were favorable) and rewards higher risk students who are making positive strides to lower their risk*.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

*Context-based filtering - loans will be assigned not based on preference of the user, but on the conditions surrounding the user (ability to re-pay, grades, need ect.)*

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Student loans difer from other types of loans i.e car, home. Applicants are younger and their economic conditions are almost entirely a result of their family circumstances. A student from a family with means, a good school, will be labeled 'low-risk' for a loan whereas a student from a more difficult situation will be considered 'high-risk' due to no fault of their own. This puts in place a barrier to pursue higher education, potentially perpetuation a cycle of privelege and poverty. At the same time, no one benefits form issueing loans to people who can not repay them or have the interest in higher education.

Any loan system has to balance the abilty to re-pay the loan while being able to put it to use in a meaningful way. One way to do this would be to issue a grant to "high risk" students with good grades no obligation to pay back. Also, the system should be dynamic, adjusting rates, dollar amounts with the performance of the student.

